# Results

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

from vessel_manoeuvring_models.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from vessel_manoeuvring_models.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
matplotlib.rcParams["figure.figsize"] = (15,4)

from myst_nb import glue
from vessel_manoeuvring_models.symbols import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models.system_equations import *

from IPython.display import display, Math, Latex, Markdown
from sympy.physics.vector.printing import vpprint, vlatex

from vessel_manoeuvring_models.parameters import df_parameters
p = df_parameters["symbol"]

# Read configs:
conf_path = os.path.join("../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmm_names = global_variables["vmms"]

ship_data = catalog.load("wpcc.ship_data")

In [ ]:
runs_meta_data = catalog.load("wpcc.runs_meta_data")
model_test_ids = [int(value) for value in model_test_ids['wpcc']]
mask = runs_meta_data.index.isin(model_test_ids)
runs_meta_data = runs_meta_data.loc[mask].copy()

In [ ]:
runs_meta_data

In [ ]:
runs_meta_data['comment'].unique()

In [ ]:
def mirror(df):
    
    df['y0']*=-1
    df['psi']*=-1
    df['v']*=-1
    df['r']*=-1
    df['v1d']*=-1
    df['r1d']*=-1
    df['delta']*=-1
    
    return df
    

In [ ]:
descriptions = ['ZigZag10/10','ZigZag20/20','Turning circle']
run_groups = runs_meta_data.groupby(by='description')

dataframes_all = {}

for description in descriptions:
            
    dataframes = dataframes_all[description] = {}
    df_ = run_groups.get_group(description)
    for id,row in df_.iterrows():
                
        data_ = catalog.load(f"wpcc.{ id }.data_ek_smooth")
        
        if 'PS' in row['comment']:
            data_ = mirror(data_)
                
        dataframes[id] = data_
        
        

In [ ]:
for description, dataframes in dataframes_all.items():
    
    track_plots(dataframes, lpp=ship_data['L'], beam=ship_data['B'])
    plot(dataframes)

In [ ]:
descriptions = ['ZigZag10/10','ZigZag20/20','Turning circle']
mask = runs_meta_data['description'].isin(descriptions)
runs_meta_data = runs_meta_data.loc[mask]
mask = runs_meta_data.duplicated(subset='description', keep='last')
runs = runs_meta_data.loc[~mask].copy()
runs

In [ ]:
dataframes_all = {}
vmm_name = 'vmm_abkowitz_simple'

dataframes_all = {}

for id,row in runs.iterrows():
    
    dataframes = dataframes_all[row['description']] = {}
    
    data_smooth = catalog.load(f"{ id }.data_ek_smooth")
    data_sim = catalog.load(f"{ vmm_name }.motion_regression.joined.{ id }.data_resimulate")
                
    dataframes['model test'] = data_smooth
    dataframes[vmm_name] = data_sim

In [ ]:
for description, dataframes in dataframes_all.items():
    
    track_plots(dataframes, lpp=ship_data['L'], beam=ship_data['B'], plot_boats=False)
    plot(dataframes, keys=['psi','u','v','r'])


In [ ]:
%reload_kedro
df_ = catalog.load(f"kvlcc2.{id}.raw_data")